## Check DB!

In [ ]:
import os
import yaml
import subprocess
import json

MELTANO_TEST_DIR='meltano-test'

def quick_subprocess(command):
    subprocess.run(command.split(' '))
    

In [ ]:
!pip install git+https://gitlab.com/meltano/tap-gitlab.git

In [ ]:
quick_subprocess('tap-gitlab -c gitlab-config.json')

In [ ]:
with open("gitlab-config.json", 'r') as f:
    tap_gitlab_config = json.loads(f.read())

print(json.dumps(tap_gitlab_config, indent=4))

## So what's the problem?
#### - how do you know what config is needed? Specificiation doesn't require disclosing so you're at the whims of the connector maintainer
#### - how do you know what you can select from the tap in terms of entities/fields?
#### - how do you manage secrets when you *HAVE* to pass the configs in a file?

In [ ]:
quick_subprocess(f'meltano init {MELTANO_TEST_DIR}')

In [ ]:
os.chdir(MELTANO_TEST_DIR)

In [ ]:
def get_meltano_config():
    with open("meltano.yml", 'r') as f:
        return yaml.load(f, Loader=yaml.Loader)

print(yaml.dump(get_meltano_config()))

In [ ]:
quick_subprocess('meltano discover extractors')

In [ ]:
quick_subprocess('meltano add extractor tap-gitlab')

In [ ]:
print(yaml.dump(get_meltano_config()))

In [ ]:
quick_subprocess('meltano invoke tap-gitlab --help')

In [ ]:
quick_subprocess('tap-gitlab --help')

In [ ]:
quick_subprocess('meltano config tap-gitlab list')
## THIS IS A MELTANO SPECIFIC THING!!!!!!

In [ ]:
subprocess.run(['meltano', 'config', 'tap-gitlab', 'set', 'projects', '"meltano/meltano meltano/tap-gitlab"'])

In [ ]:
print(yaml.dump(get_meltano_config()))

In [ ]:
quick_subprocess('meltano config tap-gitlab set start_date 2021-07-18T00:00:00Z')

In [ ]:
quick_subprocess('meltano config tap-gitlab')

In [ ]:
quick_subprocess('meltano select tap-gitlab --list --all')

In [ ]:
commands = [
    'meltano select tap-gitlab commits id',
    'meltano select tap-gitlab commits project_id',
    'meltano select tap-gitlab commits created_at',
    'meltano select tap-gitlab commits author_name',
    'meltano select tap-gitlab commits message',
    'meltano select tap-gitlab tags "*"',
    'meltano select tap-gitlab --exclude "*" "*_url"'
]
for command in commands:
    quick_subprocess(command)

In [ ]:
print(yaml.dump(get_meltano_config()))

In [ ]:
quick_subprocess('meltano discover loaders')

In [ ]:
!export LDFLAGS="-L/usr/local/opt/openssl/lib"
!export CPPFLAGS="-I/usr/local/opt/openssl/include"
!meltano add loader target-postgres

In [ ]:
quick_subprocess('meltano invoke target-postgres --help')

In [ ]:
commands = [
    'meltano config target-postgres set postgres_host localhost',
    'meltano config target-postgres set postgres_port 5432',
    'meltano config target-postgres set postgres_username meltano',
    'meltano config target-postgres set postgres_password meltano',
    'meltano config target-postgres set postgres_database warehouse',
    'meltano config target-postgres set postgres_schema public'
]
for command in commands:
    quick_subprocess(command)

In [ ]:
print(yaml.dump(get_meltano_config()))

In [ ]:
quick_subprocess('meltano config target-postgres')

In [ ]:
quick_subprocess('meltano elt tap-gitlab target-postgres --job_id=gitlab-to-postgres')